# Counting the Prameters and Operations for MicroNet

Here is the plan.

You can select the version of micronet. There are two versions and the checkpoints we uploaded on github are from ver2 network,
so if you want to verify our model, please choose the network version as 'ver2'.

Our network was trained with FP-32 precision, but we will calculate our score with freebie quantization rule.
Therefore, we calculate the number of parameters and multiplication operations with the assumption of 16bit not on addition.

Our network has below operations:

- Module: `Convolution`, `AveragePool`, `FullyConnected`
- Activation: `HardSwish`, `Sigmoid`

We also used the batch normalization, but we heard that we can count except BN in terms of storage and for math operations even BN is not merged just as the code shows.

***
We calculate the math operations with our method from `Counting` as considering the sparsity.
we count the `Hard Swish` function as below:

addition: 1, multiplication: 4

Hard swish function have this formula: x * relu_6(x + 3.0) / 6.0. From your previous annoucnments,
relu_6 function have two multiplicaiton operations, so you can easily get above results.

In our counting code, actually this doesnot capture our hard swish fuction, but count with relu 6 function.
However, we only use the relu6 function in Hard swish activation, so it is proper to get exact flops count.

In [1]:
import os
import time
import argparse
import shutil
import math

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from Counting import count
import matplotlib.pyplot as plt

from Utils import *
from Models import *

### These are for Score Calculation.
- count_nonzero: count the number of nonzero parameters except BN
- micro_score: get score. Flops are already calculated with sparsity, so we multiply the non zero ratio only at params. In addition, we did not use any quantization so that we can assume freebie quantization rule. Therefore, we calculate the score with multiplying 0.5 both on multiplications and params.

In [2]:
def bitmask(net):
    num = 0 
    for module in net.parameters():
        if module.ndimension() != 1:
            num += module.numel()
    #1-bit per parameter
    return num/32

In [3]:
def micro_score(net, precision = 'Freebie'):
    input = torch.randn(1, 3, 32, 32).to(net.device)
    addflops, multflops, params = count(net, inputs=(input, ))

    #use fp-16bit
    if precision == 'Freebie':
        multflops = multflops / 2
        params = params / 2
    #add bit-mask
    params += bitmask(net)
    
    score = (params/36500000 + (addflops + multflops)/10490000000)
    #print('Non zero ratio: {}'.format(non_zero_ratio))
    print('Score: {}, flops: {}, params: {}'.format(score, addflops + multflops, params))
    return score

### Build Model

To verify our checkpoints, here is the details of checkpoints.
Here, we will verify our ver1, ver2 checkpoints. They are in the `Checkpoint_ver1` and`Checkpoint_ver2` folder, respectively.

#### Ver1
- micronet_v1.t7: version 1 model with no pruning
- micronet_v1_pr_60.t7: version 1 model with pruning 60%. --> This is our ver1 checkpoint to submit.

#### Ver2
- micronet_v2.t7: version 2 model with no pruning
- micronet_v2_pr_15.t7: version 2 model with 15% pruning
- micronet_v2_pr_30.t7: version 2 model with 30% pruning
- micronet_v2_pr_45.t7: version 2 model with 45% pruning
- micronet_v2_pr_60.t7: version 2 model with 60% pruning --> This is our ver2 checkpoint to submit.

The `micronet_v1.t7` and `micronet_v2.t7`  files have four checkpoints with dictionary type, so if you want to load the checkpoints for verification, load the checkpoints with the key <strong>'net4'.

In [4]:
transform_test = transforms.Compose([
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.5071, 0.4865, 0.4409) , (0.2673, 0.2564, 0.2762))])
testset = datasets.CIFAR100(root='./Data/CIFAR100', train=False, download=False, transform=transform_test)   
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

### Evaluate the test accuracy
We here load the final pruned network with 60%, and evaluate with <strong>CIFAR100 test dataset.

#### Micronet Version 1 (with 60% prune)

In [6]:
net1 = MicroNet(ver = 'ver1', num_classes = 100, add_se = True, Activation = 'HSwish', expansion=3)
checkpoint = torch.load('./Checkpoint_ver1/micronet_v1_pr_60.t7')
net1.load_state_dict(checkpoint, strict = False)
net1.to(net1.device)
eval_32bit(net1, test_loader)

100%|██████████| 100/100 [00:02<00:00, 41.68it/s]

Loss: 0.748 | Acc1: 80.070% | Acc5: 95.820%


(80.07, 0.7479651227593422)

#### Micronet Version 2 (with 60% prune)

### Score
Then, you can get the score.

In [7]:
ver1_score = micro_score(net1)
#ver2_score = micro_score(net2)

Count has not implemented counting method for  CrossEntropyLoss()
Register FLOP counter for module Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Register FLOP counter for module BatchNorm2d(32, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module Conv2d(32, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
Register FLOP counter for module BatchNorm2d(96, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=96, bias=False)
Register FLOP counter for module BatchNorm2d(96, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
Register FLOP counter for module BatchNorm2d(16, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
Register FLOP counter for module ReLU6(inplace)
Register

In [8]:
print('micronet version1 score: {:.4f}'.format(ver1_score))

micronet version1 score: 0.0054


## Submission

#### As above results, we would like to submit version1 checkpoint. Then, Our score is <strong> `0.0054`.